In [6]:
from collections import Counter
import numpy as np
from collections import defaultdict
from copy import deepcopy
import time

In [7]:

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [8]:



def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

In [11]:
#Monte Carlo Tree Search

class Node():
    def __init__(self):
        self.initialize_game()

    def initialize_game(self):
        self.board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
        self.player_turn = 1
    
    def check_end(self):
        if four_in_a_row(self.board, 1):
            # Alice won
            return self.player_turn
        elif four_in_a_row(self.board, -1):
            # Bob won
            return -self.player_turn
        elif self.board.all():
            #IT IS A TIE
            return 0
        return None

    def is_terminal(self):
        if self.check_end() != None:
            return True
        else :
            return False
    
    def valid_moves(self): 
        return [n for n in range(NUM_COLUMNS) if self.board[n, COLUMN_HEIGHT - 1] == 0]

    def play(self, column):
        (index,) = next((i for i, v in np.ndenumerate(self.board[column]) if v == 0))
        new_node = deepcopy(self)
        new_node.board[column, index] = self.player_turn
        new_node.player_turn = -new_node.player_turn
        return new_node

class MonteCarloTreeSearchNode():
    def __init__(self, state, parent=None, parent_action=None):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children = []
        self._number_of_visits = 0
        self._results = defaultdict(int)
        self._results[1] = 0
        self._results[-1] = 0
        self._untried_actions = self.untried_actions()
        return

    def untried_actions(self):
        self._untried_actions = self.state.valid_moves()
        return self._untried_actions

    def quality(self):
        wins = self._results[1]
        loses = self._results[-1]
        return wins - loses

    def n_visits(self):
        return self._number_of_visits

    def expand(self):
	
        action = self._untried_actions.pop()
        next_state = self.state.play(action)
        child_node = MonteCarloTreeSearchNode(
		    next_state, parent=self, parent_action=action)

        self.children.append(child_node)
        return child_node 

    def is_terminal_node(self):
        return self.state.is_terminal()

    def rollout(self):
        current_rollout_state = self.state
    
        while not current_rollout_state.is_terminal():
        
            possible_moves = current_rollout_state.valid_moves()
        
            action = possible_moves[np.random.randint(len(possible_moves))]
            current_rollout_state = current_rollout_state.play(action)
        return current_rollout_state.check_end()

    def backpropagate(self, result):
        self._number_of_visits += 1.
        self._results[result] += 1.
        if self.parent:
            self.parent.backpropagate(result)

    def best_child(self, c_param=0.1):
        choices_weights = [(c.quality() / c.n_visits()) + c_param * np.sqrt((2 * np.log(self.n_visits()) / c.n_visits())) for c in self.children]
        return self.children[np.argmax(choices_weights)]

    def _tree_policy(self):

        current_node = self
        while not current_node.is_terminal_node():
        
            if len(current_node._untried_actions) != 0:
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

    def best_action(self):
        montecarlo_samples = 100
        for i in range(montecarlo_samples):
		
            v = self._tree_policy()
            reward = v.rollout()
            v.backpropagate(reward)
	
        return self.best_child(c_param=0.)

def playmc():
    turn =1
    initial_node = Node()
    root = MonteCarloTreeSearchNode(initial_node)
    start_time = time.time()
    selected_node = root.best_action()
    print('Turn',turn, 'Player:', root.state.player_turn)
    print(selected_node.state.board)
    turn +=1
    print("%.2f seconds" % (time.time() - start_time))
    while not selected_node.is_terminal_node():
        start_time = time.time()
        selected_node = selected_node.best_action()
        print('Turn', turn , 'Player:',selected_node.state.player_turn)
        print(selected_node.state.board)
        turn +=1
        print("%.2f seconds" % (time.time() - start_time))
    result = selected_node.state.check_end()
    if result != None:
        if result == 1:
            print('The winner is Player 1')
        elif result == -1:
            print('The winner is Player -1')
        elif result == 0:
            print("It's a tie!")
playmc()

Turn 1 Player: 1
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]]
2.20 seconds
Turn 2 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]]
2.08 seconds
Turn 3 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  1  0  0  0]]
2.07 seconds
Turn 4 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  1  0  0  0]]
2.32 seconds
Turn 5 Player: -1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  1  0  0  0]]
1.87 seconds
Turn 6 Player: 1
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1 -1  0  0  0  0]
 [ 1 -1